In [1]:
import os
import json
import cv2
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Input, ConvLSTM2D, MaxPooling3D, TimeDistributed, Dense, Flatten, Dropout, Bidirectional, MaxPooling2D, InputLayer, BatchNormalization
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, Callback
from IPython.display import clear_output


# Data Select 📚

In [2]:
# SCVD Paths
scvd_train_dir = '/kaggle/input/smartcity-cctv-violence-detection-dataset-scvd/SCVD/SCVD_converted_sec_split/Train'
scvd_test_dir = '/kaggle/input/smartcity-cctv-violence-detection-dataset-scvd/SCVD/SCVD_converted_sec_split/Test'
scvd_classes = ['Normal', 'Violence','Weaponized']

# RWF Paths
# rwf_train_dir = '/kaggle/input/rwf2000/RWF-2000/train'
# rwf_test_dir = '/kaggle/input/rwf2000/RWF-2000/val'
# rwf_classes = ['NonFight','Fight']

# RealLife Dataset
reallife_dir = '/kaggle/input/real-life-violence-situations-dataset/'
reallife_classes = ['NonViolence','Violence']

# Hyperparameters 🔖

In [3]:
IMG_SIZE = 224
BATCH_SIZE = 8
EPOCHS = 20
LEARNING_RATE = 1e-6
FRAME_COUNT = 15
DATA_GENERATOR_BATCH_SIZE_SCVD = 1
# DATA_GENERATOR_BATCH_SIZE_RWF = 1
DATA_GENERATOR_BATCH_SIZE_REALLIFE = 1

In [4]:
!apt-get install -y ffmpeg


/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()





ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 68 not upgraded.


In [5]:
!ffmpeg -loglevel error -i '/kaggle/input/smartcity-cctv-violence-detection-dataset-scvd/SCVD/SCVD_converted/Train/Weaponized/w001_converted.avi' '/kaggle/working/weapon001.mp4'


In [6]:
!ffmpeg -loglevel error -i '/kaggle/input/smartcity-cctv-violence-detection-dataset-scvd/SCVD/SCVD_converted/Train/Violence/v001_converted.avi' '/kaggle/working/Violence001n.mp4'


In [7]:
from IPython.display import Video

Video("/kaggle/working/weapon001.mp4", embed=True,width=400, height=400)


In [8]:
Video("/kaggle/working/Violence001n.mp4", embed=True,width=400, height=400)

In [9]:
train_dataset = scvd_train_dir
val_dataset = scvd_test_dir

# Data Generator for SCVD 📗

In [10]:
# class VideoFrameGenerator_SCVD(tf.keras.utils.Sequence):
#     """
#     This function will extract all the frames i.e. 30 frames from the SCVD videos and return them in chunks of FRAME_COUNTS
#     BATCH_SIZE: Controls the number of video files processed
#     """
#     def __init__(self, video_paths, batch_size=BATCH_SIZE, frame_count=FRAME_COUNT, img_size=IMG_SIZE, shuffle=True):
#         self.video_paths = video_paths
#         self.batch_size = batch_size
#         self.frame_count = frame_count
#         self.img_size = img_size
#         self.shuffle = shuffle
#         self.on_epoch_end()

#     def __len__(self):
#         return int(np.floor(len(self.video_paths) / self.batch_size))

#     def __getitem__(self, index):
#         # Determine batch indices
#         batch_paths = self.video_paths[index * self.batch_size:(index + 1) * self.batch_size]
#         X, y = self._data_generation(batch_paths)
#         return X, y

#     def _data_generation(self, batch_paths):
#         X = []
#         y = []
        
#         for video_path, label in batch_paths:
#             frames = self._extract_frames(video_path)
#             total_frames = len(frames)
            
#             # Extract frames in chunks of `frame_count`
#             for start in range(0, total_frames - self.frame_count + 1, self.frame_count):
#                 window_frames = frames[start:start + self.frame_count]
#                 X.append(window_frames)
#                 y.append(label)
        
#         # Convert lists to numpy arrays
#         X = np.array(X)
#         y = np.array(y)
        
#         return X, y

#     def _extract_frames(self, video_path):
#         frames = []
#         cap = cv2.VideoCapture(video_path)
#         total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
# #         fps = cap.get(cv2.CAP_PROP_FPS)
        
#         for i in range(total_frames):
#             ret, frame = cap.read()
#             if not ret:
#                 break
#             frame = cv2.resize(frame, (self.img_size, self.img_size))
#             frame = frame / 255.0
#             frames.append(frame)
        
#         cap.release()
#         return np.array(frames)
    
#     def on_epoch_end(self):
#         if self.shuffle:
#             np.random.shuffle(self.video_paths)

# Data Generator For RealLife Dataset 📒

# Read Frames 🖊️

In [11]:
import cv2

# Modify your function to use cv2.resize
def read_frames(root_folder, arr, each_nth=10):
    videos = [] 
    for j in range(len(arr)):
        clear_output()
        print(np.round(100 * j / len(arr), 3))
        
        vcap = cv2.VideoCapture(root_folder + arr[j])
        success = True
  
        frames = []
        cnt = 0
        while success:
            try:
                success, image = vcap.read()
                cnt += 1
                if success and cnt % each_nth == 0:
                    image = cv2.resize(image, (128, 192))  # Use cv2.resize
                    frames.append(image)
            except Exception as e:
                print(e)
        videos.append(frames)
    
    return videos


In [12]:
def select_frames(frames_arr , n=10):
    videos=[]
    for i in range(len(frames_arr)):
        frames=[]
        for t in np.linspace(0, len(frames_arr[i])-1, num=n):
            frames.append(frames_arr[i][int(t)])
        videos.append(frames)
        
    videos = np.array(videos)
    print(videos.shape)
    return videos

In [13]:
label_mapping = {
    'Normal': 0,
    'Violence': 1,
    'Weaponized': 2
}


In [14]:

# Define the root folder where your video datasets are stored
root_folder = '/kaggle/input/smartcity-cctv-violence-detection-dataset-scvd/SCVD/SCVD_converted/Train/'

# Create an array to hold video paths and labels
video_list = []
labels = []

# Add videos from Normal class
normal_videos = os.listdir(os.path.join(root_folder, 'Normal'))
video_list.extend([os.path.join('Normal', vid) for vid in normal_videos])
labels.extend([label_mapping['Normal']] * len(normal_videos))

# Add videos from Violence class
violence_videos = os.listdir(os.path.join(root_folder, 'Violence'))
video_list.extend([os.path.join('Violence', vid) for vid in violence_videos])
labels.extend([label_mapping['Violence']] * len(violence_videos))

# Add videos from Weaponized class
weaponized_videos = os.listdir(os.path.join(root_folder, 'Weaponized'))
video_list.extend([os.path.join('Weaponized', vid) for vid in weaponized_videos])
labels.extend([label_mapping['Weaponized']] * len(weaponized_videos))


In [15]:
videos = read_frames(root_folder, video_list, each_nth=10)


99.749


In [16]:
# Select 10 frames from each video
selected_frames = select_frames(videos, n=15)


(399, 15, 192, 128, 3)


# 🪓Splitting🪓

In [17]:
from sklearn.model_selection import train_test_split

# Convert frames to numpy arrays and normalize the pixel values (e.g., between 0 and 1)
selected_frames = np.array(selected_frames) / 255.0
labels = np.array(labels)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(selected_frames, labels, test_size=0.2, random_state=42)

In [18]:
# from tensorflow.keras import layers, models
# def build_3d_cnn(input_shape):
#     model = models.Sequential()
#     model.add(layers.Conv3D(32, (3, 3, 3), activation='relu', input_shape=input_shape))
#     model.add(layers.MaxPooling3D(pool_size=(2, 2, 2)))
#     model.add(layers.Conv3D(64, (3, 3, 3), activation='relu'))
#     model.add(layers.MaxPooling3D(pool_size=(2, 2, 2)))

    
#     model.add(layers.Flatten())
#     model.add(layers.Dense(256, activation='relu'))
#     model.add(layers.Dense(3, activation='softmax'))  # 3 classes: Normal, Violence, Weaponized

#     model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
#     return model

# # Get the input shape based on the frame data
# input_shape = (selected_frames.shape[1], selected_frames.shape[2], selected_frames.shape[3], selected_frames.shape[4])
# model = build_3d_cnn(input_shape)

In [19]:
from tensorflow.keras import layers, models

def build_3d_cnn(input_shape):
    model = models.Sequential()
    
    model.add(layers.Conv3D(32, (3, 3, 3), activation='relu', input_shape=input_shape, padding='same'))
    
    # First pooling layer
    model.add(layers.MaxPooling3D(pool_size=(2, 2, 2)))

    model.add(layers.Conv3D(64, (3, 3, 3), activation='relu', padding='same'))
    
    # Second pooling layer
    model.add(layers.MaxPooling3D(pool_size=(2, 2, 2)))

    model.add(layers.Conv3D(128, (3, 3, 3), activation='relu', padding='same'))  
    model.add(layers.Conv3D(256, (3, 3, 3), activation='relu', padding='same'))

    # Last pooling layer
    model.add(layers.MaxPooling3D(pool_size=(2, 2, 2)))

    model.add(layers.Flatten())
    
    model.add(layers.Dense(512, activation='relu'))
    model.add(layers.Dense(256, activation='relu'))
    model.add(layers.Dense(3, activation='softmax'))  # 3 classes: Normal, Violence, Weaponized

    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Get the input shape based on the frame data
input_shape = (selected_frames.shape[1], selected_frames.shape[2], selected_frames.shape[3], selected_frames.shape[4])
model = build_3d_cnn(input_shape)

# Print the model summary to verify the architecture
model.summary()


/opt/conda/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv3d (Conv3D)                 │ (None, 15, 192, 128,   │         2,624 │
│                                 │ 32)                    │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling3d (MaxPooling3D)    │ (None, 7, 96, 64, 32)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3d_1 (Conv3D)               │ (None, 7, 96, 64, 64)  │        55,360 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling3d_1 (MaxPooling3D)  │ (None, 3, 48, 32, 64)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3d_2 (Conv3D)               │ (None, 3, 48, 32, 128) │       221,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3d_3 (Conv3D)               │ (None, 3, 48, 32, 256) │       884,992 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling3d_2 (MaxPooling3D)  │ (None, 1, 24, 16, 256) │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 98304)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │    50,332,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 3)              │           771 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 51,628,547 (196.95 MB)

 Trainable params: 51,628,547 (196.95 MB)

 Non-trainable params: 0 (0.00 B)

In [20]:
history = model.fit(X_train, y_train, epochs=20, batch_size=8, validation_split=0.2)


Epoch 1/20


I0000 00:00:1728914529.594878    1707 service.cc:145] XLA service 0x79c774003fa0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1728914529.594931    1707 service.cc:153]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
2024-10-14 14:02:14.774729: E external/local_xla/xla/service/slow_operation_alarm.cc:65] Trying algorithm eng0{} for conv (f32[64,32,3,3,3]{4,3,2,1,0}, u8[0]{0}) custom-call(f32[8,32,7,96,64]{4,3,2,1,0}, f32[8,64,7,96,64]{4,3,2,1,0}), window={size=3x3x3 pad=1_1x1_1x1_1}, dim_labels=bf012_oi012->bf012, custom_call_target="__cudnn$convBackwardFilter", backend_config={"operation_queue_id":"0","wait_on_operation_queues":[],"cudnn_conv_backend_config":{"conv_result_scale":1,"activation_mode":"kNone","side_input_scale":0,"leakyrelu_alpha":0}} is taking a while...
2024-10-14 14:02:14.832805: E external/local_xla/xla/service/slow_operation_alarm.cc:133] The operation took 1.058259588s
Trying algorithm 

 1/32 ━━━━━━━━━━━━━━━━━━━━ 4:51 9s/step - accuracy: 0.6250 - loss: 1.0811

I0000 00:00:1728914536.948421    1707 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


32/32 ━━━━━━━━━━━━━━━━━━━━ 21s 359ms/step - accuracy: 0.4655 - loss: 2.6778 - val_accuracy: 0.5156 - val_loss: 1.0809
Epoch 2/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 5s 141ms/step - accuracy: 0.4850 - loss: 1.0775 - val_accuracy: 0.5156 - val_loss: 1.0724
Epoch 3/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 5s 141ms/step - accuracy: 0.4282 - loss: 1.0806 - val_accuracy: 0.5156 - val_loss: 1.0064
Epoch 4/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 5s 140ms/step - accuracy: 0.4995 - loss: 1.0498 - val_accuracy: 0.5156 - val_loss: 1.0384
Epoch 5/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 5s 141ms/step - accuracy: 0.4302 - loss: 1.0747 - val_accuracy: 0.5156 - val_loss: 1.1361
Epoch 6/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 5s 141ms/step - accuracy: 0.4897 - loss: 1.0238 - val_accuracy: 0.5156 - val_loss: 1.0156
Epoch 7/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 5s 141ms/step - accuracy: 0.4847 - loss: 1.0331 - val_accuracy: 0.5938 - val_loss: 0.9901
Epoch 8/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 5s 140ms/step - accuracy: 0.4554 - loss: 1.0364 - val_accuracy: 0.5156 - val

In [21]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f'Test accuracy: {test_acc}')

3/3 ━━━━━━━━━━━━━━━━━━━━ 6s 1s/step - accuracy: 0.7094 - loss: 1.9365
Test accuracy: 0.699999988079071


In [22]:
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.layers import LSTM

base_model = MobileNet(weights='imagenet', include_top=False, input_shape=(128, 128, 3))


for layer in base_model.layers:
    layer.trainable = False


model = Sequential()
model.add(TimeDistributed(base_model, input_shape=(20, 128, 128, 3)))
model.add(TimeDistributed(GlobalAveragePooling2D())) 
model.add(TimeDistributed(Dropout(0.5)))
model.add(TimeDistributed(BatchNormalization()))

model.add(LSTM(64, return_sequences=False))

model.add(Dense(128, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))  

model.compile(optimizer=Adam(learning_rate=1e-3), loss='binary_crossentropy', metrics=['accuracy'])

model.summary()

17225924/17225924 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/wrapper.py:27: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ time_distributed                │ ?                      │     3,228,864 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_1              │ ?                      │   0 (unbuilt) │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_2              │ ?                      │   0 (unbuilt) │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_3              │ ?                      │   0 (unbuilt) │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ ?                      │   0 (unbuilt) │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,228,864 (12.32 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 3,228,864 (12.32 MB)

In [23]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
checkpoint = ModelCheckpoint(
    '/kaggle/working/best_model.keras',  
    monitor='val_loss',
    save_best_only=True,  
    mode='min',  
)
history = model.fit(
    x=X_train,
    y= y_train,
    validation_data=(X_test,y_test),
    callbacks=[early_stopping,checkpoint],
    epochs= 30,
)

Epoch 1/30
10/10 ━━━━━━━━━━━━━━━━━━━━ 51s 1s/step - accuracy: 0.3582 - loss: 0.7573 - val_accuracy: 0.4250 - val_loss: 0.6148
Epoch 2/30
10/10 ━━━━━━━━━━━━━━━━━━━━ 4s 359ms/step - accuracy: 0.4853 - loss: 0.2405 - val_accuracy: 0.4750 - val_loss: 0.5201
Epoch 3/30
10/10 ━━━━━━━━━━━━━━━━━━━━ 4s 360ms/step - accuracy: 0.6016 - loss: -0.2730 - val_accuracy: 0.4875 - val_loss: 0.4210
Epoch 4/30
10/10 ━━━━━━━━━━━━━━━━━━━━ 4s 360ms/step - accuracy: 0.5824 - loss: -1.0505 - val_accuracy: 0.5125 - val_loss: 0.2472
Epoch 5/30
10/10 ━━━━━━━━━━━━━━━━━━━━ 4s 360ms/step - accuracy: 0.5938 - loss: -2.0688 - val_accuracy: 0.5875 - val_loss: -0.0623
Epoch 6/30
10/10 ━━━━━━━━━━━━━━━━━━━━ 4s 359ms/step - accuracy: 0.6398 - loss: -3.9584 - val_accuracy: 0.6250 - val_loss: -0.5142
Epoch 7/30
10/10 ━━━━━━━━━━━━━━━━━━━━ 4s 359ms/step - accuracy: 0.6557 - loss: -6.9021 - val_accuracy: 0.6625 - val_loss: -1.0457
Epoch 8/30
10/10 ━━━━━━━━━━━━━━━━━━━━ 4s 358ms/step - accuracy: 0.6673 - loss: -10.4026 - val_accu